In [1]:
import pandas

In [4]:
import psycopg
import pandas as pd

In [25]:
conn_params = {
    "host": "127.0.0.1",  # e.g., "localhost"
    "dbname": "postgres",  # e.g., "mydatabase"
    "user": "postgres",  # e.g., "myusername"
    "password": "postgres",  # e.g., "mypassword"
    "port": "5432"  # default port for PostgreSQL
}

In [26]:
conn = psycopg.connect(**conn_params)

In [27]:
query = "select hdate, open, high, low, close, volume, trade_count   from turtle.bars_history bh where symbol = 'AMZN' and hdate > '2024-07-01' order by hdate"

In [ ]:
with psycopg.connect(**conn_params) as conn:
    with conn.cursor() as cur:
        cur.execute(query)
        # Fetch all rows
        rows = cur.fetchall()
        # Get column names from the cursor description
        col_names = [desc.name for desc in cur.description]

dtypes = {
    "hdate": "string",
    "open": "float64",
    "high": "float64",
    "low": "float64",
    "close": "float64",
    "volume": "int64",
    "trade_count": "int64"
}

# Create a pandas DataFrame from the fetched data
df = pd.DataFrame(rows, columns=col_names).astype(dtypes)
df['hdate'] = pd.to_datetime(df['hdate'])
df = df.set_index(['hdate'])

# Display the DataFrame
print(df)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28 entries, 2024-07-02 to 2024-08-09
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   open         28 non-null     float64
 1   high         28 non-null     float64
 2   low          28 non-null     float64
 3   close        28 non-null     float64
 4   volume       28 non-null     int64  
 5   trade_count  28 non-null     int64  
dtypes: float64(4), int64(2)
memory usage: 1.5 KB


In [33]:
import pandas_ta as ta

In [34]:
df['SMA_10'] = ta.sma(df['close'], length=10)

In [ ]:
df.info()

In [36]:
# Resample to weekly OHLC data
df_weekly = df.resample('W').agg({
    'open': 'first',  # First day's open price
    'high': 'max',    # Highest price of the week
    'low': 'min',     # Lowest price of the week
    'close': 'last',   # Last day's close price
    'volume': 'sum',   # Last day's close price
    'trade_count': 'sum'   # Last day's close price
})
print(df_weekly)

              open    high     low   close     volume  trade_count
hdate                                                             
2024-07-07  197.28  200.55  195.93  200.00  117056824      1332045
2024-07-14  200.04  201.20  192.86  194.49  175571316      2247090
2024-07-21  194.56  196.62  180.11  183.13  216879535      2858323
2024-07-28  185.00  189.39  176.80  182.50  202972080      2369676
2024-08-04  183.84  190.60  160.55  167.90  326330023      3791941
2024-08-11  154.21  168.55  151.61  166.94  272525762      3272189
